**Predição ossos da mão**

In [1]:
import cv2
from ultralytics import YOLO

# Carregar o modelo YOLO
model = YOLO('models/bestHandBones.pt')

# Ler a imagem de entrada
img = cv2.imread('images/handfracture.png')

# Realizar a predição na imagem
results = model.predict(img, verbose=False)

# Mostrar a imagem com as detecções
for objeto in results:
    nomes = objeto.names
    
    for item in objeto.boxes:
        # Converte as coordenadas para inteiros
            x1, y1, x2, y2 = item.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Identifica a classe do objeto
            cls = int(item.cls[0])
            nomeClasse = nomes[cls]
            #print(nomeClasse)
            
            # Confiança da predição
            conf = round(float(item.conf[0]), 2)
            texto = f'{nomeClasse} - {conf}'
            
            if conf>=0.6:
                #print(texto)
                cv2.putText(img, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

# Salvar a imagem com as predições
cv2.imwrite('images/mao_ossos_predicao.png', img)


True

**Predição Fratura**

In [2]:
import cv2
from ultralytics import YOLO

# Carregar o modelo YOLO
model = YOLO('models/osso.pt')

# Ler a imagem de entrada
img = cv2.imread('images/handfracture.png')

# Realizar a predição na imagem
results = model.predict(img, verbose=False)

# Mostrar a imagem com as detecções
for objeto in results:
    nomes = objeto.names
    
    for item in objeto.boxes:
        # Converte as coordenadas para inteiros
            x1, y1, x2, y2 = item.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Identifica a classe do objeto
            cls = int(item.cls[0])
            nomeClasse = nomes[cls]
            #print(nomeClasse)
            
            # Confiança da predição
            conf = round(float(item.conf[0]), 2)
            texto = f'{nomeClasse} - {conf}'

            # Adiciona texto e bounding box            
            cv2.putText(img, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

# Salvar a imagem com as predições
cv2.imwrite('images/mao_fratura_predicao.png', img)

True

**Predição e coordenadas**

In [3]:
import cv2
from ultralytics import YOLO
import os

def porcentagem_area_em_comum(osso_xy, fratura_xy):
    
    # Área da fratura
    area_fratura_xy = (fratura_xy[2] - fratura_xy[0]) * (fratura_xy[3] - fratura_xy[1])  
    
    # Coletando as coordenadas da intersecção
    x1_inter = max(fratura_xy[0], osso_xy[0])
    y1_inter = max(fratura_xy[1], osso_xy[1])
    x2_inter = min(fratura_xy[2], osso_xy[2])
    y2_inter = min(fratura_xy[3], osso_xy[3])
    
    # Dimensões da interseção
    largura_inter = max(0, x2_inter - x1_inter)
    altura_inter = max(0, y2_inter - y1_inter)
    
    # Área da interseção
    area_interseccao = largura_inter * altura_inter
    
    # Porcentagem da área do bounding box da fratura contida no do osso
    porcentagem_contida = (area_interseccao / area_fratura_xy) * 100 if area_fratura_xy > 0 else 0
    
    return porcentagem_contida

# Carregando modelo de detecção dos ossos da mão
modelOssos = YOLO('models/bestHandBones.pt')

# Carregando modelo de detecção de fraturas
modelFratura = YOLO('models/osso.pt')


output_dir = 'images/classes/'

# Ler a imagem de entrada
img = cv2.imread('images/handfracture.png')

# Realizar a predição na imagem
resultadosOssos = modelOssos.predict(img, verbose=False)
resultadosFraturas = modelFratura.predict(img, verbose=False)

coordenadas_ossos = []
coordenadas_fraturas = []

# Predição dos ossos da mão
for objeto in resultadosOssos:
    nomes = objeto.names
    
    for i, item in enumerate(objeto.boxes):
        # Converte as coordenadas para inteiros
        x1, y1, x2, y2 = item.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Identifica a classe do objeto
        cls = int(item.cls[0])
        nomeClasse = nomes[cls]

        if nomeClasse in ['artefato', 'calcificacao_de_tecidos_moles']:
            break

        # Confiança da predição
        conf = round(float(item.conf[0]), 2)

        # Verifica se a confiança é maior ou igual a 0.6
        if conf >= 0.6:
            
            # Armazenando informações das coordenadas das predições
            coordenadas_ossos.append({
                'classe': nomeClasse,
                'coordenadas': (x1, y1, x2, y2) 
            }) 
            
            # Recorta a imagem da classe
            cropped_img = img[y1:y2, x1:x2]

            # Caminho para salvar a imagem da classe
            output_path = os.path.join(output_dir, f'{nomeClasse}_{i}.png')

            # Salvar a imagem da classe
            cv2.imwrite(output_path, cropped_img)

            # Adicionar a bounding box e texto na imagem original
            texto = f'{nomeClasse} - {conf}'
            #cv2.putText(img, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
            #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)
            
# Predição das fraturas
for objeto in resultadosFraturas:
    nomes = objeto.names
    
    for i, item in enumerate(objeto.boxes):
        # Converte as coordenadas para inteiros
        x1, y1, x2, y2 = item.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Identifica a classe do objeto
        cls = int(item.cls[0])
        nomeClasse = nomes[cls]

        # Confiança da predição
        conf = round(float(item.conf[0]), 2)
            
        # Armazenando informações das coordenadas das predições
        coordenadas_fraturas.append((x1, y1, x2, y2)) 
            
        # Recorta a imagem da classe
        cropped_img = img[y1:y2, x1:x2]

        # Caminho para salvar a imagem da classe
        output_path = os.path.join(output_dir, f'{nomeClasse}_{i}.png')

        # Salvar a imagem da classe
        cv2.imwrite(output_path, cropped_img)

        # Adicionar a bounding box e texto na imagem original
        texto = f'{nomeClasse} - {conf}'
        #cv2.putText(img, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
        #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

**Ordenando ossos da mão em polegar - indicador - dedo médio - anelar - dedo mínimo**

In [4]:
falanges_distais = [
    osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'] == 'falanges_distais'
]

falanges_intermediarias = [
    osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'] == 'falanges_intermediarias'
]

falanges_proximais = [
    osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'] == 'falanges_proximais'
]

ossos_metacarpais = [
    osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'].find('osso_metacarpal') != -1
]

ossos_metacarpais_ordenados = sorted(ossos_metacarpais, key=lambda coord: coord[0])
#print(ossos_metacarpais_ordenados)

falanges_distais_ordenadas = sorted(falanges_distais, key=lambda coord:coord[0])
#print(falanges_distais_ordenadas)

falanges_intermediarias_ordenadas = sorted(falanges_intermediarias, key=lambda coord:coord[0])
#print(falanges_intermediarias_ordenadas)

falanges_proximais_ordenadas = sorted(falanges_proximais, key=lambda coord: coord[0])
#print(falanges_proximais_ordenadas)

mao = 'direita'

coord_radio = [osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'] == 'radio']
coord_ulna = [osso['coordenadas'] for osso in coordenadas_ossos if osso['classe'] == 'ulna']

coordenadas_ossos_final = [{'osso': 'falange distal do polegar', 'coordenadas': falanges_distais_ordenadas[0]},
                           {'osso': 'falange distal do indicador', 'coordenadas': falanges_distais_ordenadas[1]},
                           {'osso': 'falange distal do dedo médio', 'coordenadas': falanges_distais_ordenadas[2]},
                           {'osso': 'falange distal do anelar', 'coordenadas': falanges_distais_ordenadas[3]},
                           {'osso': 'falange distal do dedo mínimo', 'coordenadas': falanges_distais_ordenadas[4]},
                           {'osso': 'falange intermediária do indicador', 'coordenadas': falanges_intermediarias_ordenadas[0]},
                           {'osso': 'falange intermediária do dedo médio', 'coordenadas': falanges_intermediarias_ordenadas[1]},
                           {'osso': 'falange intermediária do anelar', 'coordenadas': falanges_intermediarias_ordenadas[2]},
                           {'osso': 'falange intermediária do dedo mínimo', 'coordenadas': falanges_intermediarias_ordenadas[3]},
                           {'osso': 'falange proximal do polegar', 'coordenadas': falanges_proximais_ordenadas[0]},
                           {'osso': 'falange proximal do indicador', 'coordenadas': falanges_proximais_ordenadas[1]},
                           {'osso': 'falange proximal do dedo médio', 'coordenadas': falanges_proximais_ordenadas[2]},
                           {'osso': 'falange proximal do anelar', 'coordenadas': falanges_proximais_ordenadas[3]},
                           {'osso': 'falange proximal do dedo mínimo', 'coordenadas': falanges_proximais_ordenadas[4]},
                           {'osso': 'metacarpal do polegar', 'coordenadas': ossos_metacarpais_ordenados[0]},
                           {'osso': 'metacarpal do indicador', 'coordenadas': ossos_metacarpais_ordenados[1]},
                           {'osso': 'metacarpal do dedo médio', 'coordenadas': ossos_metacarpais_ordenados[2]},
                           {'osso': 'metacarpal do anelar', 'coordenadas': ossos_metacarpais_ordenados[3]},
                           {'osso': 'metacarpal do dedo mínimo', 'coordenadas': ossos_metacarpais_ordenados[4]},
                           {'osso': 'rádio', 'coordenadas': coord_radio[0]},
                           {'osso': 'ulna', 'coordenadas': coord_ulna[0]}
                          ]
print(coordenadas_ossos_final)

if coordenadas_ossos_final[0]['coordenadas'][0] > coordenadas_ossos_final[4]['coordenadas'][0]:
    mao = 'esquerda'
    coordenadas_ossos_final = [{'osso': 'falange distal do polegar', 'coordenadas': falanges_distais_ordenadas[4]},
                               {'osso': 'falange distal do indicador', 'coordenadas': falanges_distais_ordenadas[3]},
                               {'osso': 'falange distal do dedo médio', 'coordenadas': falanges_distais_ordenadas[2]},
                               {'osso': 'falange distal do anelar', 'coordenadas': falanges_distais_ordenadas[1]},
                               {'osso': 'falange distal do dedo mínimo', 'coordenadas': falanges_distais_ordenadas[0]},
                               {'osso': 'falange intermediaria do indicador', 'coordenadas': falanges_intermediarias_ordenadas[3]},
                               {'osso': 'falange intermediaria do dedo médio', 'coordenadas': falanges_intermediarias_ordenadas[2]},
                               {'osso': 'falange intermediaria do anelar', 'coordenadas': falanges_intermediarias_ordenadas[1]},
                               {'osso': 'falange intermediaria do dedo mínimo', 'coordenadas': falanges_intermediarias_ordenadas[0]},
                               {'osso': 'falange proximal do polegar', 'coordenadas': falanges_proximais_ordenadas[4]},
                               {'osso': 'falange proximal do indicador', 'coordenadas': falanges_proximais_ordenadas[3]},
                               {'osso': 'falange proximal do dedo médio', 'coordenadas': falanges_proximais_ordenadas[2]},
                               {'osso': 'falange proximal do anelar', 'coordenadas': falanges_proximais_ordenadas[1]},
                               {'osso': 'falange proximal do dedo mínimo', 'coordenadas': falanges_proximais_ordenadas[0]},
                               {'osso': 'matacarpal do polegar', 'coordenadas': ossos_metacarpais_ordenados[4]},
                               {'osso': 'metacarpal do indicador', 'coordenadas': ossos_metacarpais_ordenados[3]},
                               {'osso': 'metacarpal do dedo médio', 'coordenadas': ossos_metacarpais_ordenados[2]},
                               {'osso': 'metacarpal do anelar', 'coordenadas': ossos_metacarpais_ordenados[1]},
                               {'osso': 'metacarpal do dedo mínimo', 'coordenadas': ossos_metacarpais_ordenados[0]},
                               {'osso': 'rádio', 'coordenadas': coord_radio[0]},
                               {'osso': 'ulna', 'coordenadas': coord_ulna[0]}
                              ]

porcentagem_area = []
for fratura in coordenadas_fraturas:
    for osso in coordenadas_ossos_final:
        resultado = porcentagem_area_em_comum(osso['coordenadas'], fratura)
        porcentagem_area.append({'osso': osso['osso'], 'porcentagemInter': resultado})

print(porcentagem_area)

porcentagem_maior = {'mao': mao,'osso': 'indefinido', 'porcentagem': 0}
for dic_osso in porcentagem_area:
    if dic_osso['porcentagemInter'] > porcentagem_maior['porcentagem']:
        porcentagem_maior['osso'] = dic_osso['osso']
        porcentagem_maior['porcentagem'] = dic_osso['porcentagemInter']

print(porcentagem_maior)

[{'osso': 'falange distal do polegar', 'coordenadas': (131, 264, 197, 326)}, {'osso': 'falange distal do indicador', 'coordenadas': (252, 129, 297, 176)}, {'osso': 'falange distal do dedo médio', 'coordenadas': (313, 97, 358, 148)}, {'osso': 'falange distal do anelar', 'coordenadas': (389, 117, 435, 168)}, {'osso': 'falange distal do dedo mínimo', 'coordenadas': (463, 174, 503, 221)}, {'osso': 'falange intermediária do indicador', 'coordenadas': (246, 172, 300, 240)}, {'osso': 'falange intermediária do dedo médio', 'coordenadas': (306, 146, 360, 223)}, {'osso': 'falange intermediária do anelar', 'coordenadas': (376, 165, 430, 241)}, {'osso': 'falange intermediária do dedo mínimo', 'coordenadas': (443, 214, 498, 275)}, {'osso': 'falange proximal do polegar', 'coordenadas': (141, 319, 217, 412)}, {'osso': 'falange proximal do indicador', 'coordenadas': (244, 235, 320, 348)}, {'osso': 'falange proximal do dedo médio', 'coordenadas': (306, 220, 378, 333)}, {'osso': 'falange proximal do ane

In [5]:
mao = porcentagem_maior['mao']
ossoFraturado = porcentagem_maior['osso']

if ossoFraturado.find('falange' or 'ulna') != -1:
    print(f"Fratura na {ossoFraturado}, mão {mao}.")
else:
    print(f"Fratura no {ossoFraturado}, mão {mao}.")

Fratura no metacarpal do indicador, mão direita.


**Laudo**
<p>Laudo exemplo presente em laudo/laudo_exemplo.doc</p>

In [6]:
import shutil

laudo_exemplo = "laudo/laudo_exemplo.docx"

laudo_copia = "laudo/laudo_gerado.docx"

# Salva uma cópia do arquivo exemplo
shutil.copy(laudo_exemplo, laudo_copia)

'laudo/laudo_gerado.docx'

In [8]:
from docx import Document

# Abrindo laudo
laudo = Document(laudo_copia)

# Texto referente ao osso fraturado
texto_fratura = f'Fratura no {ossoFraturado}'

# Texto para estruturas sem fratura
texto_normal = 'Estruturas ósseas normais.'

# Metacarpos 
laudo.paragraphs[2].text = texto_fratura if texto_fratura.find('metacarpal') != -1 else texto_normal
    
# Falanges Proximais
laudo.paragraphs[4].text = texto_fratura if texto_fratura.find('proximal') != -1 else texto_normal

# Falanges Intermediárias
laudo.paragraphs[6].text = texto_fratura if texto_fratura.find('intermediária') != -1 else texto_normal    

# Falanges Distais
laudo.paragraphs[8].text = texto_fratura if texto_fratura.find('distal') != -1 else texto_normal

# Radio
laudo.paragraphs[10].text = texto_fratura if texto_fratura.find('rádio') != -1 else texto_normal

# Ulna
laudo.paragraphs[12].text = texto_fratura if texto_fratura.find('ulna') != -1 else texto_normal

# Salvando alterações
laudo.save(laudo_copia)